Tarefa
Utilize o dataset "Titanic - Machine Learning from Disaster" para realizar essa tarefa: https://www.kaggle.com/competitions/titanic/data

In [1]:
#Cria sessão Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

In [2]:
#Instância Spark
spark = SparkSession.builder.appName('Curso Pyspark') \
         .config('spark.sql.repl.eagerEval.enabled', True) \
         .getOrCreate()      

In [3]:
# lendo um arquivo parquet
df = spark.read.csv('./DATASETS/Desafio/titanic.csv',sep=',', header=True)

In [4]:
df.show(20, truncate=0)

+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22  |1    |0    |A/5 21171       |7.25   |NULL |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38  |1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26  |0    |0    |STON/O2. 3101282|7.925  |NULL |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)           |female|35  |1  

In [47]:
# Qual é a proporção de sobreviventes em relação ao total de passageiros do Titanic?

(
    df
    .select(F.avg(col("Survived")))
)

avg(Survived)
0.3838383838383838


In [23]:
# Qual é a média de idade dos passageiros que não sobreviveram
(
    df
    .filter(col("Survived") == 0)
    .select(F.avg(col("Age")))
)

avg(Age)
30.62617924528302


In [29]:
# Quantos passageiros da terceira classe sobreviveram ao naufrágio?
(
    df
    .filter((col("Pclass") == 3) & (col("Survived") == 1))
    .count()
)

119

In [46]:
# Qual a porcentagem de mulheres sobreviventes em relação ao total de mulheres a bordo do Titanic?
# mulheres sobreviventes = 233
# total de mulheres = 314
(
    # df
    # .filter((col("Sex") == "female") & (col("Survived") == 1)).count()
    # /
    # df
    # .filter(col("Sex") == "female").count()  
    
    df.filter(col("Survived") == 1)
    .agg(F.round(F.avg(col("Survived"))*100, 2)
         .alias("mulheres_sobreviventes_%"))
)

mulheres_sobreviventes_%
74.2


In [55]:
# Qual foi o preço médio da passagem dos passageiros que sobreviveram ao naufrágio?
(
    df.filter(col("Survived") == 1)
    .agg(F.round(F.avg(col("Fare")), 2)
         .alias("media_passagem_passageiros_sobreviventes_$"))
)

media_passagem_passageiros_sobreviventes_$
48.4


In [51]:
# Qual a quantidade de passageiros que embarcaram no Titanic?
(
    df
    .select('*')
    .count()
)

891

In [72]:
# Qual a proporção de passageiros do sexo masculino em relação ao total de passageiros?
#577
(
    df.filter(col("Sex") == "male").count() / df.count()
    * 100
)

64.75869809203144

In [73]:
# Qual a proporção de passageiros da terceira classe que sobreviveram?
(   
    df.filter(col("Pclass") == 3)
    .agg(F.round(F.avg(col("Survived"))*100, 2)
         .alias("terceira_classe_sobreviventes_%"))
)

terceira_classe_sobreviventes_%
24.24


In [76]:
df2 = spark.read.csv('./DATASETS/Desafio/Mall_Customers.csv',sep=',', header=True)
df2.show(20, truncate=0)

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|1         |Male  |19 |15                |39                    |
|2         |Male  |21 |15                |81                    |
|3         |Female|20 |16                |6                     |
|4         |Female|23 |16                |77                    |
|5         |Female|31 |17                |40                    |
|6         |Female|22 |17                |76                    |
|7         |Female|35 |18                |6                     |
|8         |Female|23 |18                |94                    |
|9         |Male  |64 |19                |3                     |
|10        |Female|30 |19                |72                    |
|11        |Male  |67 |19                |14                    |
|12        |Female|35 |19                |99                    |
|13       

In [77]:
# Qual é a média de idade dos clientes no Shopping?
(
    df2
    .agg(F.avg(col("Age"))
         .alias("média_idade_clientes"))
)

média_idade_clientes
38.85


In [78]:
# Quantas mulheres são clientes do Shopping?
(
    df2
    .filter(col("Gender") == "Female")
    .count()
)

112

In [80]:
# Qual é a média de renda anual dos clientes?
(
    df2
    .agg(F.avg(col("Annual Income (k$)"))
         .alias("média_renda_anual_clientes"))
)

média_renda_anual_clientes
60.56


In [81]:
# Qual é o desvio padrão da idade dos clientes?
(
    df2.select(F.stddev(col("Age")))
)

stddev(Age)
13.96900733155888


In [82]:
# Quantos clientes têm uma renda anual superior a 80 mil?
(
    df2.filter(col("Annual Income (k$)") > 80)
    .count()   
)

38

In [86]:
# Qual é a moda da pontuação de gastos dos clientes?
(
    df2
    #agrupa resultados iguais
    .groupBy(col("Spending Score (1-100)"))
    # faz a contagem de cada resultado
    .count()
    # ordena do maior para o menor
    .orderBy("count", ascending=False)
    # pega o primeiro resultado (o que tem maior número (MODA))
    .first()["Spending Score (1-100)"]
)

'42'